CH2 Data Embedding

HomeWork2.1 Tokenize  
Using BPE,byte pair encoding

In [19]:
import tiktoken as tken
from importlib.metadata import version

print(version("tiktoken"))

0.5.1


In [20]:
tokenizer = tken.get_encoding("gpt2")  #初始化tiktoken載入的model編碼方案 >> GPT2
text = "Akwirw ier"
unknown_vocab = tokenizer.encode(text)
print("token id:",unknown_vocab)
unknown_vocab_ID = tokenizer.decode(unknown_vocab)
print("token docode", unknown_vocab_ID)
for index in unknown_vocab:
    # print(index)
    unknown_index = tokenizer.decode([index])
    print(index, unknown_index)


token id: [33901, 86, 343, 86, 220, 959]
token docode Akwirw ier
33901 Ak
86 w
343 ir
86 w
220  
959 ier


HW2.2 SlidingWindows  

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
import tiktoken as tken


class GPTDataset_v1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.targeet_ids = []
        token_ids = tokenizer.encode(text)

        for i in range(0, len(token_ids)-max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+1+max_length] 
            self.input_ids.append(torch.tensor(input_chunk))
            self.targeet_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):   # 輸入指定位置的資料時，回傳相應的input與target
        return self.input_ids[index], self.targeet_ids[index]
    

def crate_dataloader_v1(text, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tken.get_encoding("gpt2")
    dataset = GPTDataset_v1(text, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=0
        )
    return dataloader


with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text=f.read()

dataloader = crate_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter= iter(dataloader)
inputs, targets = next(data_iter)
print("Token ID:\n", inputs)
print("\nInputs shape:\n", inputs.shape)



Token ID:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


Embedding to torch (token and position)  
for this case we will enbedding inputs to 256 dimenstion.

In [22]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim) #初始化設定embedding layer
token_embeddings = token_embedding_layer(inputs) #輸入需嵌入的資料
print(token_embeddings.shape)


torch.Size([8, 4, 256])


Embedding position   

In [23]:
max_length = 4
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


Conbine token ID and position >>> Input Data

In [24]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


Data Embedding whole process

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import tiktoken as tken


class GPTDataset_v1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.targeet_ids = []
        token_ids = tokenizer.encode(text)

        for i in range(0, len(token_ids)-max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+1+max_length] 
            self.input_ids.append(torch.tensor(input_chunk))
            self.targeet_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):   # 輸入指定位置的資料時，回傳相應的input與target
        return self.input_ids[index], self.targeet_ids[index]
    

def crate_dataloader_v1(text, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tken.get_encoding("gpt2")
    dataset = GPTDataset_v1(text, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=0
        )
    return dataloader

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text=f.read()

dataloader = crate_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter= iter(dataloader)
inputs, targets = next(data_iter)

vocab_size = 50257
output_dim = 256
torch.manual_seed(123)

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim) #初始化設定embedding layer
token_embeddings = token_embedding_layer(inputs) #輸入需嵌入的資料
print(token_embeddings.shape)

max_length = 4
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
torch.Size([4, 256])
torch.Size([8, 4, 256])
